In [1]:
import sys
sys.path.append("../../models/Multi-Body")
from multidataset import MultiDataset
import numpy as np
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from multimodel import Net as Model
from utils import accuracy
import sklearn
from sklearn.metrics import roc_auc_score, accuracy_score
import matplotlib.pyplot as plt
import os
import json

In [2]:
all_models = [f for f in os.listdir('../../models/Multi-Body/trained_models') if '_best' in f]
print('\n'.join(all_models))

MultiBody8-Subjettiness_massTrue_ptTrue_best
MultiBody8-Subjettiness_massTrue_ptFalse_best
MultiBody8-Subjettiness_massTrue_ptTrue_best_no_fourth_layer
MultiBody8-Subjettiness_massTrue_ptTrue_best_tau_x_1


In [3]:
#Testing path
test_path = '../../datasets/n-subjettiness_data/test_all.npy'

In [4]:
def eval2(model, data):
    data = torch.tensor(data, dtype = torch.float32).cuda()
    return model.forward(data)

In [5]:
#Parameters to change, can change model_type to 'best' or 'last'
for modelname in all_models:
    model_dict = json.load(open('../../models/Multi-Body/trained_model_dicts/' + modelname.replace("_best", "") + ".json"))
    use_jet_pt = model_dict['use_pt']
    use_jet_mass = model_dict['use_mass']
    tau_x_1 = model_dict['tau_x_1']
    N = model_dict['N']
    # model_type = 'best'
    extra_name = model_dict['label']
    if extra_name != '' and not extra_name.startswith('_'):
        extra_name = '_' + extra_name
    if tau_x_1:
        extra_name += '_tau_x_1'
    hidden = list(map(int, model_dict['nodes'].split(',')))


    #model path
    model_path = '../../models/Multi-Body/trained_models/MultiBody' + str(N) + '-Subjettiness_mass' +\
                 str(use_jet_mass) + '_pt' + str(use_jet_pt) + '_best' + extra_name


    #loading model
    model = Model(N, use_jet_pt, use_jet_mass, tau_x_1, hidden).cuda().float()
    model.load_state_dict(torch.load(model_path))
    model.eval()
    
    #Loading testing dataset
    test_set = MultiDataset(test_path, N, use_jet_pt, use_jet_mass, tau_x_1)
    data,labels = test_set[:]
    with torch.no_grad():
        results = eval2(model,data).cpu().numpy()
    auc = roc_auc_score(labels[:,0], results[:,0])*100
    accuracy = accuracy_score(labels[:,0], results[:,0].round())*100
    print("{}\t ROC-AUC: {:.4f}%, Accuracy: {:.4f}%".format(modelname, auc, accuracy))

MultiBody8-Subjettiness_massTrue_ptTrue_best	 ROC-AUC: 97.9805%, Accuracy: 92.8149%
MultiBody8-Subjettiness_massTrue_ptFalse_best	 ROC-AUC: 97.9736%, Accuracy: 92.8002%
MultiBody8-Subjettiness_massTrue_ptTrue_best_no_fourth_layer	 ROC-AUC: 97.9835%, Accuracy: 92.8299%
MultiBody8-Subjettiness_massTrue_ptTrue_best_tau_x_1	 ROC-AUC: 97.5527%, Accuracy: 92.1232%
